In [0]:
from keras.layers import GlobalAveragePooling2D, Multiply, Dense
from keras import backend as K

def SqueezeExcite(x, ratio=16, name=''):
    nb_chan = K.int_shape(x)[-1]

    y = GlobalAveragePooling2D(name='{}_se_avg'.format(name))(x)
    y = Dense(nb_chan // ratio, activation='relu', name='{}_se_den1'.format(name))(y)
    y = Dense(nb_chan, activation='sigmoid', name='{}_se_den2'.format(name))(y)

    y = Multiply(name='{}_se_mul'.format(name))([x, y])
    return y

In [0]:
import keras
from keras.datasets import mnist
from keras.models import Sequential  
from keras.layers import Dense, Dropout, Flatten ,Conv2D, MaxPooling2D,Input
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Model 
from keras import backend as K
from keras.regularizers import l2
from keras.optimizers import *

In [0]:
def conv_block(n_class=10, init_lr = 0.01):
    # layer 1
    inputs = Input((28,28,1))
    conv1 = Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),activation = 'relu',strides=(4,4), padding='same')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = SqueezeExcite(conv1, ratio=16, name='conv1')
    pool1 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv1)
    
    # layer 2
    conv2 = Conv2D(filters=256,kernel_size=(5,5),activation ='relu',strides=(1,1), padding='same')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = SqueezeExcite(conv2, ratio=16, name='conv2')
    pool2 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv2)
    
    # layer 3
    conv3 = Conv2D(filters=512,kernel_size=(3,3),activation = 'relu',strides=(4,4), padding='same')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = SqueezeExcite(conv3, ratio=16, name='conv3')
    pool3 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv1)
    
    # layer 4
    conv4 = Conv2D(filters=1024,kernel_size=(3,3),activation = 'relu',strides=(4,4), padding='same')(pool3)
    conv4 = Dropout(0.2)(conv4)
    conv4 = SqueezeExcite(conv4, ratio=16, name='conv4')
    
    # layer 5
    conv5 = Conv2D(filters=1024,kernel_size=(3,3),activation = 'relu',strides=(4,4), padding='same')(conv4)
    conv5 = Dropout(0.2)(conv5)
    conv5 = SqueezeExcite(conv5, ratio=16, name='conv5')
    pool5 = MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same')(conv5)
    
    # layer 6
    conv6 = Flatten() (pool5)
    conv6 = Dense(3072) (conv6)
    conv6 = Activation('relu') (conv6)
    conv6 = Dropout(0.4)(conv6)
    
    
    # last o/p 
    conv7 = Dense(n_class)(conv6)
    conv7 = Activation('softmax')(conv7)
    
    model = Model(input=inputs, output=conv7)
    
    model.compile(loss="categorical_crossentropy", optimizer= Adam(lr=init_lr) , metrics=['accuracy'] )
    
    
    return model

In [6]:
model = conv_block()
print(model.summary)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



<bound method Network.summary of <keras.engine.training.Model object at 0x7fa765a2feb8>>


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`


In [0]:
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(y_test.shape)
img_rows, img_cols = 28, 28
nb_class = 10
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train/= 255
x_test/= 255
y_train = np_utils.to_categorical(y_train, nb_class)
y_test = np_utils.to_categorical(y_test, nb_class)
gen = ImageDataGenerator(rotation_range = 8,width_shift_range=0.08,shear_range= 0.3, height_shift_range= 0.08,zoom_range=0.08)
test_gen = ImageDataGenerator()
print(x_test.shape)                                                                                                                                         
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)
train_generator = gen.flow(x_train,y_train,batch_size =4)
test_generator = test_gen.flow(x_test,y_test,batch_size = 4)
model.fit_generator(train_generator,steps_per_epoch =60000//64, epochs=50,validation_data= test_generator , validation_steps= 10000//64)

11493376/11490434 [==============================] - 0s 0us/step
(10000,)
(10000, 28, 28, 1)
(10000, 10)
(60000, 28, 28, 1)
(60000, 10)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





937/937 [==============================] - 352s 376ms/step - loss: 2.3246 - acc: 0.1123 - val_loss: 2.3043 - val_acc: 0.1010
Epoch 2/50
937/937 [==============================] - 352s 375ms/step - loss: 2.3047 - acc: 0.1115 - val_loss: 2.3035 - val_acc: 0.1170
Epoch 3/50
937/937 [==============================] - 349s 373ms/step - loss: 2.3062 - acc: 0.1083 - val_loss: 2.3036 - val_acc: 0.0978
Epoch 4/50
937/937 [==============================] - 349s 373ms/step - loss: 2.3036 - acc: 0.1107 - val_loss: 2.2990 - val_acc: 0.1330
Epoch 5/50
937/937 [==============================] - 354s 378ms/step - loss: 2.3060 - acc: 0.1027 - val_loss: 2.3029 - val_acc: 0.1250
Epoch 6/50
937/937 [==============================] - 352s 375ms/step - loss: 2.30

In [0]:
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import np_utils
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(y_test.shape)
img_rows, img_cols = 28, 28
nb_class = 10
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train/= 255
x_test/= 255
y_train = np_utils.to_categorical(y_train, nb_class)
y_test = np_utils.to_categorical(y_test, nb_class)
gen = ImageDataGenerator(rotation_range = 8,width_shift_range=0.08,shear_range= 0.3, height_shift_range= 0.08,zoom_range=0.08)
test_gen = ImageDataGenerator()
print(x_test.shape)                                                                                                                                         
print(y_test.shape)
print(x_train.shape)
print(y_train.shape)
train_generator = gen.flow(x_train,y_train,batch_size =4)
test_generator = test_gen.flow(x_test,y_test,batch_size = 4)
model.fit_generator(train_generator,steps_per_epoch =60000//64, epochs=100,validation_data= test_generator , validation_steps= 10000//64)

(10000,)
(10000, 28, 28, 1)
(10000, 10)
(60000, 28, 28, 1)
(60000, 10)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/100





813/937 [=========================>....] - ETA: 48s - loss: 14.4763 - acc: 0.1006